
<font size="5">**<center> ĐỒ ÁN CUỐI KỲ </center>** </font>

<font size="4">**<center> KHOA HỌC DỮ LIỆU</center>** </font>

# I/ Thông tin nhóm:

| STT |       Tên       |   MSSV  |
|-----|:---------------:|--------:|
|  1  |   Thái Bá Sơn   | 1712732 |
|  2  | Huỳnh Công Sinh | 1712724 |

# II/ Khai thác dữ liệu:

In [1]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd


Dữ liệu khai thác từ: [đây](https://www.nasdaq.com/) 

Cấu trúc câu lệnh khai thác: 'https://www.nasdaq.com/market-activity/stocks/' + 'stock_code' + '/historical'


# I/ Dữ liệu

### Thông tin về dữ liệu:


Trong sổ tay này, chúng ta sẽ xem xét dữ liệu từ thị trường chứng khoán, đặc biệt là một số cổ phiếu công nghệ. Sử dụng pandas để lấy thông tin cổ phiếu, hình dung các khía cạnh khác nhau của nó và cuối cùng ta sẽ xem xét một số cách phân tích rủi ro của cổ phiếu, dựa trên lịch sử hoạt động trước đó của nó. Chúng tôi cũng sẽ dự đoán giá cổ phiếu trong tương lai thông qua phương pháp Long Short Term Memory (LSTM)! 

**Nguồn:**

---

# II/ Khám phá dữ liệu:

### Import

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM


sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

### Đọc dữ liệu từ file csv vào dataframe

In [ ]:
df = pd.read_csv("stock_data.csv", sep = '\t')
df.head()

### Thông tin thuộc tính:

In [ ]:
num_rows, num_cols = df.shape
print("Dữ liệu có {} dòng, {} cột.".format(num_rows, num_cols))

In [ ]:
df.info()

---

## Tiền xử lý:

- Kiểm tra dữ thông tin dữ liệu: kiểu dữ liệu của các thuộc tính, các cột có bị thiếu giá trị hay không?...

In [ ]:
#xem thông tin data:
df.info()

if df.isnull().values.any() == False:
    print("=> Không có ô giá trị nào bị thiếu.")
else:
    print("Dữ liệu bị thiếu.")

- Định dạng lại kiểu dữ liệu cho phù hợp.

In [ ]:
df = pd.read_csv("stock_data.csv", sep = '\t')

#Định dạng lại cột Date sang kiểu dữ liệu datetime
df["Date"] = pd.to_datetime(df["Date"])
#Xóa khoảng trắng trong dữ liệu:
df.columns = df.columns.str.replace(' ', '') 
df.replace(' ', '', regex=True, inplace = True)

#Chuyển cột có ký hiệu tiền tệ ($) sang dạng datetime:
df['Close/Last'] = df['Close/Last'].str.replace('$', '').astype(float)
df['Open'] = df['Open'].str.replace('$', '').astype(float)
df['High'] = df['High'].str.replace('$', '').astype(float)
df['Low'] = df['Low'].str.replace('$', '').astype(float)

#Sắp xếp cổ phiếu tăng dần theo ngày:
df.sort_values(['Company', 'Date'], ascending=[True, True], inplace = True)

company_name = list(df["Company"].unique())

- Thêm ngày bị thiếu:

In [ ]:
data = pd.DataFrame(columns=df.columns)
companies = df['Company'].unique()
for company in companies:
    one_company_df = df[df['Company'] == company]
    one_company_df['Date'] = pd.to_datetime(one_company_df['Date'])
    one_company_df = one_company_df.sort_values(by=['Date'], ascending=True)
    one_company_df.set_index('Date', inplace=True)
    one_company_df = one_company_df.resample('D').bfill().reset_index()
    one_company_df = one_company_df.sort_values(by=['Date'], ascending=False).reset_index()
    data = data.append(one_company_df)
data = data.drop('index', axis=1).reset_index(drop=True)

- Dữ liệu sau khi tiền xử lý:

In [ ]:
data.info()
data.head()

In [ ]:
# Summary Stats
data.describe()

In [ ]:
100 * data.isnull().sum() / len(df)

In [ ]:
sns.kdeplot(df['Close/Last'], shade=True)
plt.show()

# III/ Đặt câu hỏi:
- 1.

- 2. 

- 3.

- 4. 

- 5. 


---

# IV/ Tìm câu trả lời:

- **<font color=red>Kiểm chứng model LSTM áp dụng cho giá cổ phiếu của Alphabet bằng dữ liệu ta có. Từ đó dự đoán giá cổ phiếu cho ngày tiếp theo sau ngày cuối trong dữ liệu? </font>**

In [3]:
def get_records_in_time_period(df, start_date, end_date, company_name=None):
    '''
    Hàm get_records_in_time_period sẽ lấy các records của công ty trong một khoảng thời gian từ một dataframe
    Tham số df: dataframe nguồn chứa các records.
    Tham số start_date: mốc thời gian bắt đầu.
    Tham số end_date: mốc thời gian kết thúc.
    Tham số company_name: tên công ty cần lấy record, mặc định None sẽ lấy toàn bộ.
    Output: một dataframe chứa các records với tất cả các cột dữ liệu.
    '''
    if company_name is None:
        return df.loc[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    else:
        company_df = df[df['Company'] == company_name]
        time_period_filter = (company_df['Date'] >= start_date) & (company_df['Date'] <= end_date)
        return company_df.loc[time_period_filter]

k = get_records_in_time_period(data, 'Apple', '2019-12-1', '2020-12-1')

NameError: name 'data' is not defined

In [4]:
def lstm_model(df, company_name, input_len=100):
    '''
    Hàm lstm_model sẽ tạo một LSTM model đã qua việc train với 80% lượng dữ liệu ta có.
    Tham số df: dataframe chứa dữ liệu để tạo model.
    
    '''
    
    stock = df[df['Company'] == company_name]['Close/Last'].values
    stock = stock.reshape(len(stock), 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_stock = scaler.fit_transform(stock)

    train_values = scaled_stock[:int(len(stock) * 0.8)]
    X_train, Y_train = [], []

    for i in range(input_len, len(train_values)):
        X_train.append(train_values[i - 100:i])
        Y_train.append(train_values[i])

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=1, epochs=1)

    return model


In [ ]:
data[data['Company'] == 'Amazon']

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplots_adjust(top=1.25, bottom=1.2)

for i, company in enumerate(company_name, 1):
    plt.subplot(2, 3, i)
    data[data['Company'] == company]['Close/Last'].plot(linewidth=2.0)
    plt.ylabel('Close/Last')
    plt.xlabel(None)
    plt.title(f"{company_name[i - 1]}")

---

---

- **<font color=red> 2. Tạo mô hình huấn luyện để dự đoán giá</font>**

Để tạo mô hình huấn luyện, bước đầu tiên ta cần phải chuẩn bị dữ liệu cho mô hình. Ở đây, ta sẽ làm việc với cột giá trị stock lúc đóng phiên giao dịch của Alphabet. Sau đó chuẩn hóa dữ liệu bằng `MinMaxScaler` của sklearn. 

In [ ]:
alpha_stock = df[df['Company'] == 'Alphabet']['Close/Last'].values
alpha_stock = alpha_stock.reshape(len(alpha_stock), 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_stock = scaler.fit_transform(alpha_stock)
scaled_stock

Với dữ liệu này, ta lấy 20% dòng cuối để làm giá trị test, còn lại ta dùng cho việc huấn luyện mô hình.

In [ ]:
train_values = scaled_stock[:int(len(alpha_stock) * 0.8)]
train_values

Sau đó, tạo ra tập X_train và Y_train.
- Với Y_train, ta lấy giá trị stock của một ngày.
- Với X_train, ta lấy giá trị của 60 dòng trước ngày đó.

In [ ]:
X_train, Y_train = [], []

for i in range(120, len(train_values)):
    X_train.append(train_values[i - 120:i])
    Y_train.append(train_values[i])

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

Bắt đầu xây dựng mô hình huấn luyện.

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, batch_size=1, epochs=1)

Sau khi tạo và huấn luyện mô hình cho tập huấn luyện, ta tạo dữ liệu cho tập test.

In [ ]:
test_values = scaled_stock[int(len(alpha_stock) * 0.8) - 120:]

In [ ]:
X_test = []
Y_test = alpha_stock[int(len(alpha_stock) * 0.8):, :]

for i in range(120, len(test_values)):
    X_test.append(test_values[i - 120:i])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))


Áp dụng mô hình cho tập dữ liệu test.

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Tính độ lỗi RMSE cho tập dự đoán và tập test.

In [ ]:
rmse = np.sqrt(np.mean(((predictions - Y_test) ** 2)))
rmse